# LangChain Retrieval Augmented Generation Example 2

[Build a Retrieval Augmented Generation (RAG) App: Part 2](https://python.langchain.com/docs/tutorials/qa_chat_history/)

Extends the implementation to accommodate conversation-style interactions and multi-step retrieval processes. This would require us to incoporating historical messages.

Set up environment
* Setup project home 
* Load OpenAI API key
* Load LangSmith key

In [3]:
import dotenv
import sys
from pathlib import Path

## Setup Environment
sys.path.append(Path.cwd().parent) # Append project home to system path
dotenv.load_dotenv() # Load .env

True

## Define RAG Components

In [12]:
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain_core.vectorstores import InMemoryVectorStore

# Connect to chat model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# Connect to embedding
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Instantiate vector store
vector_store = InMemoryVectorStore(embeddings)

## Document Loader

In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

## Document Splitter

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Splits text into chunks of 1000 characters with a 200-character overlap to maintain context between chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

## Store Documents

In [13]:
document_ids = vector_store.add_documents(documents=all_splits)

## RAG Workflow

In [15]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)